In [60]:
import pennylane as qml
import torch
from torch import nn
from kan import KAN
import pennylane as qml
from pennylane import numpy as np

n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)
    qml.layer(lambda w: [qml.Rot(w[i, 0], w[i, 1], w[i, 2], wires=i) for i in range(n_qubits)], weights.shape[0])(weights)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_layer(inputs, weights):
    return quantum_circuit(inputs, weights)

def generate_tensor(seed, size):
    """
    주어진 시드와 크기에 맞게 torch.Tensor를 생성합니다.

    Args:
        seed (int): 시드 값
        size (tuple): 생성할 텐서의 크기

    Returns:
        torch.Tensor: 생성된 텐서
    """
    torch.manual_seed(seed)
    return torch.randn(size)

class RNN_block(nn.Module):
    def __init__(self, input_size, **kwargs):
        super(RNN_block, self).__init__()
        self.linear = nn.Linear(input_size*2,input_size)
    def forward(self,input,hidden):
        hidden = torch.concat([input,hidden],dim=1)
        output = self.linear(hidden)
        return output,output[:,0]
        
    


class RNN_layer(nn.Module):
    def __init__(self,input_size,output_size,num_layers):
        """_RNN layer 만든거_

        Args:
            input_size (_int_): _input feature의 개수_
            output_size (_int_): _output feature의 개수_
            num_layers (_int_): _필요한 RNN layer 수_
        """
        super(RNN_layer, self).__init__()
        KAN_model = KAN([input_size,2*input_size+1,input_size*2-1],grid=1)
        self.input_size = input_size
        self.output_size = output_size
        self.num_layer = num_layers
        
        
        ## QNE 수행할 Linear layer
        self.QNE_layer = KAN_model 
        
        ## Ansatz parameter
        self.ansatz_params_1 = nn.Parameter(torch.rand([8,8]),requires_grad=True)
        self.ansatz_params_2 = nn.Parameter(torch.rand([8,8]),requires_grad=True)
        self.rnn_layer = RNN_block(input_size)

        
    def forward(self,inputs):
        """_summary_

        Args:
            inputs (_torch tensor_): _(batch,seq_len,feature_size)_
        """
        batch = inputs.shape[0]
        initial_t = generate_tensor(30,[inputs.shape[0],inputs.shape[2]])
        inputs = inputs.permute(1, 0, 2)
        ## inputs  = (seq_len,batch,feature_size)
        hidden,y_list = self.rnn_layer(inputs[0],initial_t)
        for input in inputs[1:]:
            hidden,y = self.rnn_layer(input,hidden)

            y_list = torch.concat([y_list,y])
        y_list = torch.reshape(y_list,[batch,-1])
        return y_list
        
        
        
        
        
        
    



In [61]:
import torch
from torch import nn
import pennylane as qml
from pennylane import numpy as np

def test_rnn_layer():
    # Parameters for the RNN layer
    input_size = 8
    output_size = 8
    num_layers = 2

    # Initialize the RNN layer
    rnn_layer = RNN_layer(input_size, output_size, num_layers)

    # Generate dummy input data
    batch_size = 5
    seq_len = 10
    feature_size = input_size
    inputs = torch.randn(batch_size, seq_len, feature_size)

    # Forward pass through the RNN layer
    output = rnn_layer(inputs)

    # Print the output shape and the output itself
    print("Output shape:", output.shape)
    print("Output:", output)

test_rnn_layer()


Output shape: torch.Size([5, 10])
Output: tensor([[ 0.5289, -0.9233, -1.1201, -0.9498,  0.3260, -1.0280, -0.3502,  0.1797,
          0.6081,  0.2400],
        [-0.1470, -0.4489, -0.3574, -0.3699,  0.0362,  0.2395, -0.8719, -0.0908,
         -0.0704,  0.5883],
        [ 0.4971, -0.3949,  0.7621, -0.6622, -0.1307, -1.0103,  0.2518, -0.8066,
         -0.0654, -0.3134],
        [ 0.5257, -0.2239,  0.1284, -0.5881,  0.4930, -0.1688,  0.3405, -0.7374,
         -1.0509,  0.0868],
        [ 0.2708, -0.5211, -0.0813, -0.0014, -0.0368,  0.0295,  0.1484, -1.0277,
         -0.7533,  0.2138]], grad_fn=<ViewBackward0>)
